# ***COVID-19 DASHBOARD***

### ***Coronavirus disease (COVID-19) is an infectious disease caused by a newly discovered coronavirus***

### ***Most people infected with the COVID-19 virus will experience mild to moderate respiratory illness and recover without requiring special treatment.  Older people, and those with underlying medical problems like cardiovascular disease, diabetes, chronic respiratory disease, and cancer are more likely to develop serious illness***
### ***The best way to prevent and slow down transmission is be well informed about the COVID-19 virus, the disease it causes and how it spreads. Protect yourself and others from infection by washing your hands or using an alcohol based rub frequently and not touching your face***
### ***The COVID-19 virus spreads primarily through droplets of saliva or discharge from the nose when an infected person coughs or sneezes, so it’s important that you also practice respiratory etiquette (for example, by coughing into a flexed elbow)***
### ***At this time, there are no specific vaccines or treatments for COVID-19. However, there are many ongoing clinical trials evaluating potential treatments. WHO will continue to provide updated information as soon as clinical findings become available***

### ***(Information provided by WHO : https://www.who.int/health-topics/coronavirus#tab=tab_1)***

In [14]:
import folium
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas.util.testing as tm
import plotly.graph_objects as go
from IPython.display import display, clear_output


plt.style.use("seaborn")

In [15]:
def freshData():
    death_records = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
    confirmed_records = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
    recovered_records = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
    cases_country_records = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')
    
    death_records.columns = map(str.lower, death_records.columns)
    confirmed_records.columns = map(str.lower, confirmed_records.columns)
    recovered_records.columns = map(str.lower, recovered_records.columns)
    cases_country_records.columns = map(str.lower, cases_country_records.columns)
    return death_records, confirmed_records, recovered_records, cases_country_records

In [16]:
death_records, confirmed_records, recovered_records, cases_country_records = freshData()

In [17]:
# death_records.head()
# confirmed_records.head()
# recovered_records.head()
# cases_country_records.head()

set(death_records.columns) == set(confirmed_records.columns) == set(recovered_records.columns)

True

In [18]:
def highlightColumns1(x):
    r = "background-color : red"
    g = "background-color : green"
    p = "background-color : pink"
    y = "background-color : yellow"
    gr = "background-color : gray"
    df = pd.DataFrame("", index = x.index, columns = x.columns)
    df.iloc[:, 0] = y
    df.iloc[:, 4] = p
    df.iloc[:, 5] = r
    df.iloc[:, 6] = g
    df.iloc[:, 7] = gr
    return df

def highlightColumns2(x):
    r = "background-color : red"
    g = "background-color : green"
    p = "background-color : pink"
    y = "background-color : yellow"
    gr = "background-color : gray"
    df = pd.DataFrame("", index = x.index, columns = x.columns)
    df.iloc[:, 0] = y
    df.iloc[:, 2] = p
    df.iloc[:, 3] = r
    df.iloc[:, 4] = g
    df.iloc[:, 5] = gr
    return df

In [19]:
# sorted_country = cases_country_records.sort_values("confirmed", ascending = False)
# sorted_country.head(10).style.apply(highlightColumns1, axis = None)

## ***DATA OF TOP 20 COUNTRIES***

In [20]:
sorted_country = cases_country_records.sort_values("confirmed", ascending = False)
usefull_data = pd.DataFrame({"Country" : sorted_country["country_region"].tolist(), "Date" : sorted_country["last_update"].tolist(), "Confirmed" : sorted_country["confirmed"].tolist(), "Deaths" : sorted_country["deaths"].tolist(), "Recovered" : sorted_country["recovered"].tolist(), "Active cases" : sorted_country["active"].tolist()})
usefull_data = usefull_data[["Country", "Date", "Confirmed", "Deaths", "Recovered", "Active cases"]]
usefull_data.head(20).style.apply(highlightColumns2, axis = None)

,Country,Date,Confirmed,Deaths,Recovered,Active cases
0,US,2020-05-19 14:32:25,1.51099e+06,90432,283178,1.13706e+06
1,Russia,2020-05-19 14:32:25,299941,2837,76130,220974
2,Brazil,2020-05-19 14:32:25,257396,16941,100459,139996
3,United Kingdom,2020-05-19 14:32:25,247709,34876,1090,211743
4,Spain,2020-05-19 14:32:25,231606,27709,150376,53521
5,Italy,2020-05-19 14:32:25,225886,32007,127326,66553
6,France,2020-05-19 14:32:25,180051,28242,61843,89966
7,Germany,2020-05-19 14:32:25,177289,8067,155681,13541
8,Turkey,2020-05-19 14:32:25,150593,4171,111577,34845
9,Iran,2020-05-19 14:32:25,124603,7119,97173,20311


## ***SELECT ANY COUNTRY TO CHECK THEIR GRAPH***

In [21]:
def on_change(change):
    with out:
        if change["new"] != "Select Any Country":  
            if change["type"] == "change" and change["name"] == "value":
                clear_output()
                countryTotalPlot(change["new"])
                # print(change['new'])
        else:
            print("Select Any Country")
            
def countryTotalPlot(country_name):
    death_records, confirmed_records, recovered_records, cases_country_records = freshData()
    dates = death_records.columns.tolist()[4:]
    deaths = death_records.loc[death_records["country/region"] == country_name].iloc[:,4:].values.tolist()[0]
    confirmed = confirmed_records.loc[confirmed_records["country/region"] == country_name].iloc[:,4:].values.tolist()[0]
    recovered = recovered_records.loc[recovered_records["country/region"] == country_name].iloc[:,4:].values.tolist()[0]
    active = [i-j-k for i, j, k in zip(confirmed, recovered, deaths)]
    
    dates.reverse(); deaths.reverse(); confirmed.reverse(); recovered.reverse(); active.reverse()
    dates = [dates[i] for i in range(0, len(dates), 10)]
    deaths = [deaths[i] for i in range(0, len(deaths), 10)]
    confirmed = [confirmed[i] for i in range(0, len(confirmed), 10)]
    active = [active[i] for i in range(0, len(active), 10)]
    recovered = [recovered[i] for i in range(0, len(recovered), 10)]
    dates.reverse(); deaths.reverse(); confirmed.reverse(); recovered.reverse(); active.reverse()

    n = 20
    plt.plot(dates, confirmed, label = "Confirmed cases (Active + Recovered + Deaths)", color = "black", marker = "p", linewidth = 2)
    plt.plot(dates, active, label = "Active cases", color = "blue", marker = "*", linewidth = 2)
    plt.plot(dates, recovered, label = "Recovered cases", color = "green", marker = "o", linewidth = 2)
    plt.plot(dates, deaths, label = "Deaths", color = "red", marker = ">", linewidth = 2)
    plt.xlabel("Dates", size = n)
    plt.ylabel("Cases", size = n)
    plt.title("Covid-19 Cases In " + country_name, size = n)
    plt.legend(loc = "upper left", prop = {"size" : n})
    plt.rcParams["figure.figsize"] = (16, 10)
    plt.show()

lst = cases_country_records["country_region"].tolist()
lst.sort()
lst.insert(0, "Select Any Country")
countries = widgets.Dropdown(options = lst)
out = widgets.Output()
        
display(countries)
display(out)
countries.observe(on_change)

Dropdown(options=('Select Any Country', 'Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and…

Output()

## ***CONFIRMED COVID-19 CASES IN TOP 20 COUNTRIES***

In [22]:
fig = px.scatter(sorted_country.head(20), x = "country_region", y = "confirmed", size = "confirmed", color = "country_region", hover_name = "country_region", size_max = 60)
fig.update_layout(title = "Covid-19 Confirmed Cases In Each Country (Top 20, Confirmed = Active + Recovered + Deaths)", xaxis_title = "Countries", yaxis_title = "Number of confirmed cases")
fig.show()

## ***ACTIVE COVID-19 CASES IN TOP 20 COUNTRIES***

In [23]:
fig = px.scatter(sorted_country.head(20), x = "country_region", y = "active", size = "active", color = "country_region", hover_name = "country_region", size_max = 60)
fig.update_layout(title = "Covid-19 Active Cases In Each Country (Top 20)", xaxis_title = "Countries", yaxis_title = "Number of active cases")
fig.show()

## ***SCORES OF TOP 20 COUNTRIES IN CURING COVID-19 DISEASE***

In [24]:
def countryCurePerformance():
    temp = sorted_country.head(20)
    countryNames = temp["country_region"].tolist()
    performance = [];
    confirmed = temp["confirmed"].tolist(); 
    recovered = temp["recovered"].tolist()
    for i, j in zip(confirmed, recovered):
        performance.append((j/(i)*100))
    df = pd.DataFrame({"country_names" : countryNames, "performance" : performance, "confirmed" : confirmed, "recovered" : recovered}).sort_values("performance", ascending = False)
    fig = px.scatter(df.head(20), x = "country_names", y = "performance", size = "performance", color = "country_names", hover_name = "country_names", size_max = 30)
    fig.update_layout(title = "Country Scores In Curing Covid-19 Disease (Top 20)", xaxis_title = "Countries", yaxis_title = "Score(%)")
    fig.show()
    
countryCurePerformance()

## ***DEATH RATE DUE TO COVID-19 IN TOP 20 COUNTRIES***

In [27]:
def countryMortalityRate():
    temp = sorted_country.head(20)
    countryNames = temp["country_region"].tolist()
    mortality_rate = temp["mortality_rate"].tolist()
    df = pd.DataFrame({"country_names" : countryNames, "mortality_rate" : mortality_rate}).sort_values("mortality_rate", ascending = False)
    fig = px.scatter(df.head(20), x = "country_names", y = "mortality_rate", size = "mortality_rate", color = "country_names", hover_name = "country_names", size_max = 30)
    fig.update_layout(title = "Death Rate Due To Covid-19 (Top 20)", xaxis_title = "Countries", yaxis_title = "Death Rate")
    fig.show()
    
countryMortalityRate()

## ***AFFECTED AREAS DUE TO COVID-19 ON THE WORLD MAP***

In [26]:
world_map = folium.Map(location = [11, 0], zoom_start = 2, max_zoom = 3, min_zoom = 2)
for i in range(len(confirmed_records)):
    folium.Circle(
        location = [confirmed_records.iloc[i]["lat"], confirmed_records.iloc[i]["long"]],
        fill = True,
        radius = (int((np.log(confirmed_records.iloc[i, -1] + 1.00001))) + 2.0) * 30000,
        fill_color = "blue",
        color = "red",
        tooltip = "<div style='margin: 0; background-color: black; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_records.iloc[i]['country/region'] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_records.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_records.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death_records.iloc[i,-1]/(confirmed_records.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
    ).add_to(world_map)
    
world_map

***Created by VIJENDRA SINGH***